In [48]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
# Necessary metadata
article_id = 14096681  # this is the unique identifier of the article on figshare

url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figsharerainfall/"

In [3]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
%%time
files_to_dl = ["data.zip"]  # feel free to add other files here
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: total: 4.39 s
Wall time: 2min 26s


In [5]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: total: 13 s
Wall time: 13 s


In [6]:
os.chdir("figsharerainfall")

In [14]:
%%time
## here we are using a normal python way for merging the data 
import pandas as pd

# files = glob.glob('figsharerainfall/*.csv')
# extension = 'csv'
# all_filenames = glob.glob('*.{}'.format(extension))
# df = pd.concat((pd.read_csv(file, index_col=0)
#                 # .assign(model=re.findall(r'(?<=\/)[^_]+(?=_)', file))
#                 .assign(model=re.findall(r'^[^_]+(?=_)', file)[0])
#                 for file in all_filenames)
#               )

files = glob.glob('figsharerainfall/*.csv')
df = pd.concat((pd.read_csv(file, index_col=0)
                .assign(model = file.strip('figsharerainfall//').split('_')[0])
                for file in files))
df.to_csv("combined_data.csv")

MemoryError: Unable to allocate 1.86 GiB for an array with shape (250055452,) and data type uint64

| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Member 1    |                  |     |           |        |            |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

In [38]:
df.tail()


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
2014-12-27 12:00:00,-30.157068,-29.214659,153.125,154.375,0.554375,TaiESM1
2014-12-28 12:00:00,-30.157068,-29.214659,153.125,154.375,7.028577,TaiESM1
2014-12-29 12:00:00,-30.157068,-29.214659,153.125,154.375,0.234757,TaiESM1
2014-12-30 12:00:00,-30.157068,-29.214659,153.125,154.375,2.097459,TaiESM1
2014-12-31 12:00:00,-30.157068,-29.214659,153.125,154.375,0.548421,TaiESM1


In [37]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [39]:
# Checking the dtype of each column
df.info()
print(f"Memory usage with float64: {df.memory_usage().sum() / 1e6:.2f} MB")

<class 'pandas.core.frame.DataFrame'>
Index: 187541589 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float32
 1   lat_max        float32
 2   lon_min        float32
 3   lon_max        float32
 4   rain (mm/day)  float32
 5   model          object 
dtypes: float32(5), object(1)
memory usage: 6.3+ GB
Memory usage with float64: 6751.50 MB


In [ ]:
# Changing the dtype of numerical columns from float64 to float32 
column_name = list(df1.select_dtypes(include=['float64']))
for name in column_name:
    df[name] = df[name].astype('float32')

In [40]:
df.info()
print(f"Memory usage with float32: {df.astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

<class 'pandas.core.frame.DataFrame'>
Index: 187541589 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float32
 1   lat_max        float32
 2   lon_min        float32
 3   lon_max        float32
 4   rain (mm/day)  float32
 5   model          object 
dtypes: float32(5), object(1)
memory usage: 6.3+ GB
Memory usage with float32: 6751.50 MB


| Team Member | Operating System | RAM | Processor | Is SSD | Time taken |
|:-----------:|:----------------:|:---:|:---------:|:------:|:----------:|
| Member 1    |                  |     |           |        |            |
| Member 2    |                  |     |           |        |            |
| Member 3    |                  |     |           |        |            |
| Member 4    |                  |     |           |        |            |

Check if time can be changed to datetime object. 

In [ ]:
# df.reset_index()
# # df['dates'] = pd.to_datetime(df['dates'], format='%Y%m%d')

# Simple EDA

In [29]:
%%time
df1 = pd.read_csv("combined_data.csv")
print(df1["model"].value_counts())

combined            125027726
MPI-ESM1-2-HR         5154240
TaiESM1               3541230
NorESM2-MM            3541230
CMCC-CM2-HR4          3541230
CMCC-CM2-SR5          3541230
CMCC-ESM2             3541230
SAM0-UNICON           3541153
FGOALS-f3-L           3219300
GFDL-CM4              3219300
GFDL-ESM4             3219300
EC-Earth3-Veg-LR      3037320
MRI-ESM2-0            3037320
BCC-CSM2-MR           3035340
MIROC6                2070900
ACCESS-CM2            1932840
ACCESS-ESM1-5         1610700
INM-CM5-0             1609650
INM-CM4-8             1609650
KIOST-ESM             1287720
FGOALS-g3             1287720
MPI-ESM-1-2-HAM        966420
MPI-ESM1-2-LR          966420
NESM3                  966420
AWI-ESM-1-1-LR         966420
NorESM2-LM             919800
CanESM5                551880
BCC-ESM1               551880
observed                46020
Name: model, dtype: int64
CPU times: total: 2min 39s
Wall time: 2min 44s


In [30]:
df1.columns

Index(['lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)', 'model'], dtype='object')

In [35]:
df1.shape

(187541589, 6)

In [41]:
%%time
counts = pd.Series(dtype=int)
for chunk in pd.read_csv("combined_data.csv", chunksize=10_000_000):
    counts = counts.add(chunk["model"].value_counts(), fill_value=0)
print(counts.astype(int))

ACCESS-CM2            1932840
ACCESS-ESM1-5         1610700
AWI-ESM-1-1-LR         966420
BCC-CSM2-MR           3035340
BCC-ESM1               551880
CMCC-CM2-HR4          3541230
CMCC-CM2-SR5          3541230
CMCC-ESM2             3541230
CanESM5                551880
EC-Earth3-Veg-LR      3037320
FGOALS-f3-L           3219300
FGOALS-g3             1287720
GFDL-CM4              3219300
GFDL-ESM4             3219300
INM-CM4-8             1609650
INM-CM5-0             1609650
KIOST-ESM             1287720
MIROC6                2070900
MPI-ESM-1-2-HAM        966420
MPI-ESM1-2-HR         5154240
MPI-ESM1-2-LR          966420
MRI-ESM2-0            3037320
NESM3                  966420
NorESM2-LM             919800
NorESM2-MM            3541230
SAM0-UNICON           3541153
TaiESM1               3541230
combined            125027726
observed                46020
dtype: int32
CPU times: total: 2min 22s
Wall time: 2min 22s


In [42]:
%%time
print(df['rain (mm/day)'].min())

-3.807373e-12
CPU times: total: 1.08 s
Wall time: 1.09 s


In [43]:
%%time
df.describe()

CPU times: total: 30.4 s
Wall time: 30.7 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,1.777456e+08,1.874035e+08,1.777456e+08,1.874035e+08,1.778837e+08
mean,-3.310555e+01,-3.197907e+01,1.469056e+02,1.482144e+02,1.901831e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.588274e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.876672e-06
50%,-3.300000e+01,-3.204189e+01,1.468750e+02,1.481250e+02,6.161705e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.021314e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


In [45]:
%%time
df.isnull().sum()

CPU times: total: 21.7 s
Wall time: 21.7 s


lat_min          9795960
lat_max           138060
lon_min          9795960
lon_max           138060
rain (mm/day)    9657900
model                  0
dtype: int64

In [52]:
%%time
use_cols = ['rain (mm/day)', 'model']
df3 = pd.read_csv("combined_data.csv", usecols = use_cols)
# df3.plot.line(xlabel="Time", ylabel="Sunspots", legend=False);
df3.head()

CPU times: total: 1min 47s
Wall time: 1min 47s


,rain (mm/day),model
0,3.293256e-13,ACCESS-CM2
1,0.000000e+00,ACCESS-CM2
2,0.000000e+00,ACCESS-CM2
3,0.000000e+00,ACCESS-CM2
4,1.047658e-02,ACCESS-CM2


In [ ]:
%%time
use_cols = ['ArrDelay','DepDelay','Distance','UniqueCarrier']
df = pd.read_csv("figshareairline/combined_data.csv",usecols=use_cols)